In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [57]:
df1 = pd.read_csv("../data/Guney2016.tsv", sep="\t")
df1 = df1.drop(["Unnamed: 0"], axis=1)
df1.head()

,Disease,1,2,3,4,5,6,7,8,9,...,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022
0,kidney diseases,79663,5310.0,4306.0,760.0,81494.0,6557.0,5190.0,7225.0,6519.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,congenital hypothyroidism,7080,389434.0,169792.0,7253.0,1482.0,7038.0,6528.0,50506.0,405753.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"sarcoma, small cell",2130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mucopolysaccharidosis vi,411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pseudohypoaldosteronism,4306,1729.0,65266.0,6337.0,65125.0,6338.0,10312.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df2 = pd.read_csv("../data/mesh_disease.csv")
df2.head()

,Unnamed: 0,X,CATEGORY,MESHID,MESHTERM,QUALIFIER,QUALIFIERID,SYNONYM
0,1,1,C,D000006,"abdomen, acute",history,Q000266,Acute Abdomen
1,2,2,C,D000006,"abdomen, acute",epidemiology,Q000453,Acute Abdomen
2,3,3,C,D000006,"abdomen, acute",congenital,Q000151,Acute Abdomen
3,4,4,C,D000006,"abdomen, acute",nursing,Q000451,Acute Abdomen
4,5,5,C,D000006,"abdomen, acute",immunology,Q000276,Acute Abdomen


In [59]:
result_df2 = df2.drop_duplicates(subset=['MESHID', 'MESHTERM'], keep='first')
# result_df2 = result_df2.rename(index = list(range(4528)))
# result_df2["X"] = range(1,4529)
result_df2 = result_df2.drop(["Unnamed: 0"], axis=1)
result_df2.head()

,X,CATEGORY,MESHID,MESHTERM,QUALIFIER,QUALIFIERID,SYNONYM
0,1,C,D000006,"abdomen, acute",history,Q000266,Acute Abdomen
105,106,C,D000007,abdominal injuries,immunology,Q000276,"Injury, Abdominal"
210,211,C,D000008,abdominal neoplasms,physiopathology,Q000503,"Neoplasm, Abdominal"
327,328,C,D000012,abetalipoproteinemia,prevention & control,Q000517,"Disease, Betalipoprotein Deficiency"
803,804,C,D000013,congenital abnormalities,rehabilitation,Q000534,Birth Defect


In [60]:
print(len(df2), len(result_df2))

1657927 4528


In [61]:
meshidterm = {}

for i in result_df2["X"]:
    key = result_df2["MESHID"][i-1]
    value = result_df2["MESHTERM"][i-1]
    meshidterm[key] = value

print(meshidterm)

{'D000006': 'abdomen, acute', 'D000007': 'abdominal injuries', 'D000008': 'abdominal neoplasms', 'D000012': 'abetalipoproteinemia', 'D000013': 'congenital abnormalities', 'D000014': 'abnormalities, drug-induced', 'D000015': 'abnormalities, multiple', 'D000016': 'abnormalities, radiation-induced', 'D000022': 'abortion, spontaneous', 'D000026': 'abortion, habitual', 'D000027': 'abortion, incomplete', 'D000030': 'abortion, missed', 'D000031': 'abortion, septic', 'D000033': 'abortion, threatened', 'D000034': 'abortion, veterinary', 'D000037': 'abruptio placentae', 'D000038': 'abscess', 'D000039': 'peritonsillar abscess', 'D000051': 'acantholysis', 'D000052': 'acanthosis nigricans', 'D000126': 'achlorhydria', 'D000130': 'achondroplasia', 'D000137': 'acid-base imbalance', 'D000138': 'acidosis', 'D000140': 'acidosis, lactic', 'D000141': 'acidosis, renal tubular', 'D000142': 'acidosis, respiratory', 'D000151': 'acinetobacter infections', 'D000152': 'acne vulgaris', 'D000153': 'acne keloid', 'D

In [62]:
df3 = pd.read_csv("../data/original_disease_drug.tsv", sep = "\t")
df3.head()

,# Disease(MESH),Chemical
0,MESH:D005923,DB00564
1,MESH:D009503,DB01072
2,MESH:D016115,DB01759
3,MESH:D018476,DB00451
4,MESH:C567059,DB00641


In [63]:
print(len(df3))

466657


In [64]:
count = 0
notinlist = []

for i in range(len(df3)):
    chemical = df3["Chemical"][i]
    disease = df3["# Disease(MESH)"][i]
    disease = disease[5:]
    if(disease not in meshidterm and disease not in notinlist):
        count+=1
        notinlist.append(disease)

print(count)

2513


In [65]:
count = 0
for i in range(len(df1)):
    disease_name = df1["Disease"][i]
    for key in meshidterm:
        if(disease_name == meshidterm[key]):
            count+=1
            break
            
print("Diseases with MESH IDs:", count)
print("Total Diseases:", len(df1))

Diseases with MESH IDs: 1446
Total Diseases: 1489


In [66]:
Guney16_preprocessed = pd.merge(result_df2["MESHID"], df1,
                                how='inner', 
                                right_on=df1["Disease"], 
                                left_on=result_df2["MESHTERM"])
Guney16_preprocessed = Guney16_preprocessed.drop(["key_0", "Disease"], axis=1)
Guney16_preprocessed 

,MESHID,1,2,3,4,5,6,7,8,9,...,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022
0,D000012,4547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,D000013,374654,5820.0,9289.0,5824.0,4000.0,5828.0,3854.0,3852.0,3853.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,D000015,374654,1302.0,8626.0,2657.0,5190.0,2138.0,5192.0,5077.0,5824.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D000052,2261,3643.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,D000130,2261,1280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1441,D059327,4920,3239.0,8200.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1442,D059345,4538,2717.0,1282.0,4535.0,1471.0,51128.0,4541.0,4540.0,5654.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1443,D059446,7405,4838.0,55997.0,7547.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1444,D061085,374654,2736.0,6496.0,6469.0,3476.0,7050.0,5727.0,3897.0,9990.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
#Adding Sars cov row
sars_cov19 = ["D000086382"]                                              

In [92]:
cov19_df = pd.read_csv("../data/SARSprotein-Humanprotein.csv")
print(cov19_df.shape)
cov19_df.head()

(332, 3)


,SARS,EntrezID,Symbol
0,sars-cov2e,8546,AP3B1
1,sars-cov2e,23476,BRD4
2,sars-cov2e,6046,BRD2
3,sars-cov2e,10283,CWC27
4,sars-cov2e,124245,ZC3H18


In [98]:
len(set(target_proteins))

332

In [93]:
target_proteins = list(cov19_df["EntrezID"])
target_proteins

[8546,
 23476,
 6046,
 10283,
 124245,
 57153,
 9512,
 10897,
 481,
 34,
 2108,
 2040,
 2677,
 523,
 5714,
 7905,
 23203,
 196527,
 10531,
 10463,
 60493,
 148867,
 10426,
 79934,
 113174,
 92840,
 92105,
 89874,
 10844,
 80222,
 57142,
 51439,
 10157,
 26993,
 25980,
 28969,
 9136,
 26986,
 1459,
 1460,
 10146,
 8761,
 23367,
 25940,
 79753,
 5976,
 4343,
 9908,
 9188,
 55131,
 25873,
 51125,
 25921,
 5422,
 5557,
 5558,
 23649,
 79709,
 5318,
 161,
 2671,
 57222,
 1173,
 80273,
 6902,
 55206,
 10295,
 10270,
 23077,
 10569,
 8737,
 9898,
 219743,
 51248,
 84726,
 55833,
 24149,
 64784,
 159195,
 29066,
 23185,
 55285,
 6938,
 53938,
 54477,
 9755,
 11113,
 3281,
 5116,
 11116,
 5576,
 5566,
 5577,
 5962,
 1063,
 7088,
 7090,
 166,
 2802,
 2801,
 2804,
 56850,
 9857,
 9659,
 9662,
 23177,
 11064,
 23085,
 9648,
 79745,
 51199,
 201134,
 145282,
 8975,
 79571,
 152789,
 55755,
 10142,
 64689,
 79443,
 79078,
 11190,
 29110,
 51361,
 22981,
 2717,
 3615,
 23408,
 10204,
 382,
 10193,
 1

In [94]:
sars_cov19 = sars_cov19 + target_proteins + [np.nan]*690
len(sars_cov19)

1023

In [95]:
temp = pd.Series(sars_cov19, index = Guney16_preprocessed.columns)
Guney16_preprocessed = Guney16_preprocessed.append(temp, ignore_index=True)

In [96]:
Guney16_preprocessed.to_csv("../data/Guney16_preprocessed.csv")

In [3]:
df3 = pd.read_csv("../data/drug-gene.csv")
df3 = df3.drop(["Unnamed: 0"], axis=1)
print(df3.shape)
df3.head()

(24648, 13)


,type,ID,Name,State,Approved,Indication,Gene_Target,DB_id,name,organism,Type,entrez_id,ATC_code
0,biotech,DB00001,Lepirudin,liquid,approved,For the treatment of heparin-induced thrombocy...,F2,BE0000048,Prothrombin,Humans,Polypeptide,2147.0,B01AE02
1,biotech,DB00002,Cetuximab,liquid,approved,"Cetuximab, used in combination with irinotecan...",EGFR,BE0002100,Low affinity immunoglobulin gamma Fc region re...,Humans,Polypeptide,1956.0,L01XC06
2,biotech,DB00002,Cetuximab,liquid,approved,"Cetuximab, used in combination with irinotecan...",FCGR3B,BE0002100,Low affinity immunoglobulin gamma Fc region re...,Humans,Polypeptide,2215.0,L01XC06
3,biotech,DB00002,Cetuximab,liquid,approved,"Cetuximab, used in combination with irinotecan...",C1R,BE0002100,Low affinity immunoglobulin gamma Fc region re...,Humans,Polypeptide,715.0,L01XC06
4,biotech,DB00002,Cetuximab,liquid,approved,"Cetuximab, used in combination with irinotecan...",C1QA,BE0002100,Low affinity immunoglobulin gamma Fc region re...,Humans,Polypeptide,712.0,L01XC06


In [4]:
df3 = df3[["ID", "entrez_id"]]
print(df3.shape)
df3.head()

(24648, 2)


,ID,entrez_id
0,DB00001,2147.0
1,DB00002,1956.0
2,DB00002,2215.0
3,DB00002,715.0
4,DB00002,712.0


In [5]:
df3.to_csv("../data/drug-gene.csv")

In [20]:
df4 = pd.read_csv("../data/PPI.csv")
print(df4.shape)
df4.head()

(327924, 3)


,proteinA_entrezid,proteinB_entrezid,databases
0,9796,56992,bioplex|HIPPIE
1,7918,9240,instruct|hiunion|APID|biogrid|HINT-binary|intact
2,8233,23548,bioplex|HIPPIE
3,4899,11253,encode
4,5297,8601,bioplex


In [21]:
df4 = df4[["proteinA_entrezid", "proteinB_entrezid"]]
df4.rename(columns = {"proteinA_entrezid": "gene1", "proteinB_entrezid": "gene2"}, inplace = True)
print(df4.shape)
df4.head()

(327924, 2)


,gene1,gene2
0,9796,56992
1,7918,9240
2,8233,23548
3,4899,11253
4,5297,8601


In [22]:
df4.to_csv("../data/ppi_preprocessed.csv")